# Digital Value Realization (DVR) Metrics and Analytics

## Gold-Standard Calculation Test-Bed

### Version 4.x

### *Date: 2019-08-12*

### Kris Matson

Jira Epic ID: [DA-344](https://agrium.atlassian.net/browse/DA-344])

---

This notebook implements the DVR calculations on a production dataset with basic householding

## Digital Value Realization Metrics Definitions

Reference: VR_digital_breakout_May_20190710.pdf

### New Grower Revenue
__Hypothesis__: Is there a positive difference between revenue from digitally engaged new customers vs. non-engaged new customers?
__Computations:__
- Sum(Revenue from New Customers who are Engaged)
- Sum(Revenue from New Customers who are Non-Engaged)
  - Revenue calculated YTD for only the current year
__Intuition:__
- New Grower Revenue is expected to __increase__ throughout the current year as the number of engaged customers increases and the number of new customers increases 

### Benefit from Digital Customer Retention
__Hypothesis__: Is there a positive trend to revenue benefit from digital engagement v. non-engagement?
__Computations:__
- Count(Engaged Customers) * Avg(Annual Sales Revenue for Engaged Customers) * %(Churn Non-Engaged - Churn Engaged)
  - Assume '%' means we want the percent revenue (USD) benefit
  - Sales revenue is calculated from the prior year only
__Intuition:__
- Benefit from Digital Engagement is expected __increase__ over the current year as the number of engaged customers increases and as churn decreases throughout the year
  - Benefit will be __positive__ if churn from engaged customers is smaller than churn from non-engaged customers
  
### Increased Customer Retention
__Hypothesis__: Is Nutrien Customer Retention higher for digitally engaged customers v. non-engaged customers?
__Computations:__
- %(Non-Engaged Customer Retention - Engaged Customer Retention)
  - Retention of engaged customers is expected to be __greater than__ churn from non-engaged customers

### Increased Share of Wallet
__Hypothesis__: Does the overall Nutrien "buy" increase as a result of engagement v. non-engagement? 
__Computations:__
- Avg(Annual Revenue for Engaged Customers) - Avg(Annual Revenue for NON-Engaged Customers)
  - The denominator for the average revenue calculation is the number of unique, engaged households for the given year 
__Intuition:__
- Share of wallet is expected to be __positive__ with increased portal engagement

### Increased Agronomic Services Revenue, Engaged and Non-Engaged
__Hypothesis__: Does the Nutrien "buy" for Agronomic Services increase as a result of engagement v. non-engagement? 
__Computations:__
- YoY, YTD Comparison of Sum(Application Services Revenue for Engaged Customers)
- YoY, YTD Comparison of Sum(Application Services Revenue for NON-Engaged Customers)
  - requires shelf-level sales data
__Intuition:__
- Agronomic services revenue should be higher for the digitally engaged cohort v. the non-engaged, because of increased customer touchpoints to promote agronomic services sales.

### Definitions

#### Engagement (Digital)
- Engaged Customer: A business organization that does (or has done) business with Nutrien, with one or more users of the on-line portal acting on behalf of the business, and those user members of the business have __returned__ to the portal two or more times collectively, aggregated at the business organization level. 
  - Engagement is expected to __increase__ as digitally adopted customers return to use the portal
  - For the current analyses, once a household (aka customer) is engaged it is considered engaged for the prior year and the current year of the DVR analysis. Future versions of the analysis will consider the date of first engagement (the date of adoption == logging in for the first time and linking one or more accounts) 
  - Examples:
    - 1. An organization with a single portal user who has visited a first time to register, and then comes back to the portal two or more times, is classified as an engaged customer.
    - 2. An organization with two portal users who have both visited a first time to register, and then one comes back to the portal two or more times, is classified as an engaged customer.
    - 3. An organization with three portal users who have all visited the portal one time to register, and then none ever come back to the portal, is classified as a non-engaged customer.

#### Retention
- Retention: Count(Households with 2018 purchases who ALSO have 2019 purchases) / Count(Households with 2018 purchases)
- == Count(Households with prior year purchases who ALSO have current year purchases) / Count(Households with prior year purchases)
  - Purchases for current year are YTD based on records to-date in the data set 
  - Retention is expected to __increase__ throughout the current year as prior year accounts for a household are activated (bought against) in the current year

#### Churn
- Churn: Count(Households with prior yr purchases but no current yr purchases) / Count(Households with prior year purchases)
  - Purchases for current year are YTD based on records to-date in the data set 
  - Churn is expected to __decrease__ over the current year as prior year accounts get activated in the current year
  - Churn of engaged customers is expected to be __less than__ churn from non-engaged customers

#### New Customer:
- New Customer: (Households with 2019 account purchases but no 2018 account purchases)
  - Purchases for current year are YTD based on records to-date in the data set 
  - The new cusotmer count is expected to __increase__ over the current year as new households are acquired in the current year

---

### Assumptions, Caveats and Issues

- The label of engagement for an organization is attributed to all transaction records for that organization. In other words, transactions for organization are considered engaged for all-time if the organization is engaged at any time. Households engaged (having 2 return visits) since April 2018 have the same engagement attribution as households with 2 return visits as of yesterday. Future versions of the DVR analysis may take into account timing of first portal use as a feature explaining variation of transactional or in-portal behaviors. 


- For the current analysis we are just looking at Net Sales, not Gross Sales nor margin.


- User ID tracking in the Digital Portal was turned on in Google Analytics and Google Tag Manager August 19, 2018 and became fully functional August 20 2018. This means that usage of the portal for early users, and by association the organization they are part of, will not be tracked in GA until August 20. A partial workaround is to use the date of portal registration in Customer MDM, this gives the date of first visit.  


- The current implementation includes customers that both did and did not purchase products from CXH. A future implementation is required to isolate the CXH purchaser cohort. 


- All Methods are currently explicitly tied to 2018 and 2019 data only


---

### TODO:
date: 08/28/2019
- add CXH only cohort
- verify implemention validity of YTD and financial period calculations


In [2]:
import os
import pandas as pd
import numpy as np
import pickle

# read pickle file and output
# there is no conversion of the output
def un_pickle(pkl_fname):
    with open(pkl_fname, 'rb') as handle:
        result = pickle.load(handle)
    return result

In [3]:
# Retention, Churn, and New Customer definition getters
#
# For all methods:
# inputs: pd dataframe by Household (HH) containing:
#   HH ID: cust_ID
#   Digital Engagement of HH: cn_engaged
#   Sales variable: net_sales
#   Year and month for each transaction: fiscal_period (yyyyMM)
#   Calendar year for each total: fiscal_year (yyyy)
#
# All Methods are currently explicitly parameterized to 2018 and 2019 data and not tested for data outside this range

## Churn
# Count(distinct households with sales in prior year and no sales in current year) / 
#               Count(distinct households with sales in prior year)
# input: pd df of sales as described above
#        curr_year: int parameter for the current year for the calc
# output: float, proportion of churned customers 
# assumes: current year and prior year are valid data in fiscal_year column
#          net_sales and hh_id are valid columns in the df
def get_org_churn(df, for_year=2019):
    prev_year = for_year - 1
    # use set type for fastest operation
    hh_id_prevyr_sales = set(df.query('fiscal_year == @prev_year and net_sales != 0')['hh_id'])
    hh_id_curryr_sales = set(df.query('fiscal_year == @for_year and net_sales != 0')['hh_id']) 
    # return the number of ids last year that dont match this year / number of ids last year
    # TODO: check for div by 0
    return len(hh_id_prevyr_sales - hh_id_curryr_sales) / len(hh_id_prevyr_sales)


In [4]:
## Organization Retention
# Retention: Count(Households with previous year purchases who ALSO have purchases in the current year) / 
#              Count(Households with previous year purchases)
# input: pandas df: dataset of sales as described above
#        curr_year: int parameter for the current year for the calc
# output: float, proportion of retained customers
# assumes: current year and prior year are valid data in fiscal_year column
#          net_sales and hh_id are valid columns in the df
#
# TODO: add test for existence of year in the df
# TODO: add tests for existence of columns in the df
#
def get_org_retention(hh_df, for_year=2019, engaged=True, verbose=False):
    prev_year = for_year - 1
    # use set type for fastest operation
    hh_id_prevyr_sales = set(hh_df.query('hh_engaged == @engaged & fiscal_year == @prev_year and net_sales != 0')['hh_id'].unique())
    #print('hh_id_prevyr_sales {}'.format(hh_id_prevyr_sales))

    hh_id_curryr_sales = set(hh_df.query('hh_engaged == @engaged & fiscal_year == @for_year and net_sales != 0')['hh_id'].unique())
    #print('hh_id_curryr_sales {}'.format(hh_id_curryr_sales))

    numerator = len(hh_id_prevyr_sales.intersection(hh_id_curryr_sales))
    if verbose:
        print('numerator {}'.format(numerator))

    denominator = len(hh_id_prevyr_sales)
    if verbose:
        print('denominator {}'.format(denominator))

    # return the num ids that match from both years / num ids last year
    # TODO: check for div by 0, however it should never be zero
    org_retention =  numerator / denominator
    
    return org_retention

In [5]:
## Get New Customers:
#   pd Dataframe where (Households had current year account purchases) AND (Households with no prior YTD account purchases)
# e.g., records where the household has sales in 2019 but not in 2018 
# input: pd df of sales as described above
# output: pd dataframe of new customers
# assumes: current year and prior year are valid data in fiscal_year column
#          net_sales and hh_id are valid columns in the df
def get_new_customers(df, curr_year=2019):
    prev_year = curr_year - 1
    # ids with 2018 sales
    hh_id_prevyr_sales = list(df.query('fiscal_year == @prev_year and net_sales != 0')['hh_id'].unique())
    # this df has 2019 sales
    new_customer_df = df.query('fiscal_year == @curr_year and net_sales != 0')
    # return the df with no cust_IDs from 2018
    return new_customer_df[~new_customer_df['hh_id'].isin(hh_id_prevyr_sales)]


In [6]:
## Get Count Engaged Customers
# Calculate count distinct of engaged customers with transactions for the given year
# NOTE: Does NOT take into account dates of transactions since portal began nor the date the org first engaged
# input: pd df of sales: as described above
# output: int: length of the list of unique engaged customers       
#        curr_year: int parameter for the current year for the calc
def get_count_engaged_customers(df, for_year=2019):
    return len(list(df.query('fiscal_year == @for_year and net_sales != 0 and hh_engaged == True')['hh_id'].unique()))

In [7]:
## Get Count Non-Engaged Customers
# Calculate count distinct of NON-engaged customers with transactions for the given year
# NOTE: Does NOT take into account dates of transactions since portal began nor the date the org first engaged
# input: pd df of sales: as described above
# output: int: length of the list of unique non-engaged customers       
#        curr_year: int parameter for the current year for the calc
def get_count_non_engaged_customers(df, for_year=2019):
    return len(list(df.query('fiscal_year == @for_year and net_sales != 0 and hh_engaged == False')['hh_id'].unique()))

In [8]:
## Average Annual Revenue for Engaged Customers
# Average annual revenue for engaged customers == (annual revenue for engaged customers) / (count unique engaged customers)
# input: pd df of sales: as described above
# output: float: revenue for engaged customers since portal began
def get_avg_revenue_engaged(df, for_year=2019):
    denom = get_count_engaged_customers(df, for_year)
    if denom == 0:
        return 0
    return df.query('fiscal_year == @for_year and hh_engaged == True')['net_sales'].sum() / denom

In [9]:
## Average Annual Revenue for Non-Engaged Customers
# Average annual revenue for engaged customers == (annual revenue for non-engaged hh) / (count unique non-engaged hh)
# input: pd df of sales: as described above
# output: float: revenue for engaged customers since portal began
def get_avg_revenue_non_engaged(df, for_year=2019):
    denom = get_count_non_engaged_customers(df, for_year)
    if denom == 0:
        return 0
    return df.query('fiscal_year == @for_year and hh_engaged == False')['net_sales'].sum() / denom

## Total Annual Revenue for Engaged Customers for the given year
def get_revenue_engaged(df, for_year=2019):
    return df.query('fiscal_year == @for_year and hh_engaged == True')['net_sales'].sum()


In [9]:
# test data
# todo: add NaNs to see where fails
test_data = {'hh_id': ['A', 'B', 'B', 'C', 'D', 'E', 'B', 'C', 'D', 'F'], 
        'hh_engaged': [False, True, True, True, False, True, True, True, False, True], 
        'net_sales': [5, 10, 2, 5, 25, 1, 5, 2, 0, 5],
        'gl_shelf':  ['APPL','CHEM','APPL','APPL','APPL','CHEM','APPL','CHEM','APPL','APPL'],
        'fiscal_period': [201712, 201804, 201805, 201805, 201810, 201812, 201903, 201904, 201905, 201904],
        'fiscal_year':[2017, 2018, 2018, 2018, 2018, 2018, 2019, 2019, 2019, 2019]}
test_df = pd.DataFrame(test_data)
test_df

,hh_id,hh_engaged,net_sales,gl_shelf,fiscal_period,fiscal_year
0,A,False,5,APPL,201712,2017
1,B,True,10,CHEM,201804,2018
2,B,True,2,APPL,201805,2018
3,C,True,5,APPL,201805,2018
4,D,False,25,APPL,201810,2018
5,E,True,1,CHEM,201812,2018
6,B,True,5,APPL,201903,2019
7,C,True,2,CHEM,201904,2019
8,D,False,0,APPL,201905,2019
9,F,True,5,APPL,201904,2019


In [10]:
hh_id_prevyr_sales = set(test_df.query('fiscal_year == 2018 and net_sales != 0')['hh_id'])
hh_id_curryr_sales = set(test_df.query('fiscal_year == 2019 and net_sales != 0')['hh_id']) 
print(hh_id_prevyr_sales)
print(hh_id_curryr_sales)
print(len(hh_id_prevyr_sales - hh_id_curryr_sales))
print(len(hh_id_prevyr_sales))

{'C', 'D', 'B', 'E'}
{'C', 'B', 'F'}
2
4


In [11]:
# testing with test_df
# 
# Count(households with 2018 sales who ALSO have 2019 sales) / Count(households with 2018 sales)
# 2018 sales >> [B,C,D,E]  2019 sales [B,C,D,F]
# 3/4 overall
# 
# 2018 engaged sales >> [B,C,E]  2019 engaged sales [B,C]
# 2/3 overall
print("test org_retention")
org_retention = get_org_retention(test_df, engaged=True, verbose=True)
print(org_retention)
# retention should be 
assert(org_retention == 2./3.)

org_retention = get_org_retention(test_df, engaged=False, verbose=True)
print(org_retention)
# retention should be 
assert(org_retention == 0./1.)

# Churn
# Count(households with 2018 sales who LEFT in 2019) / Count(households with 2018 sales)
print("test org_churn")
org_churn = get_org_churn(test_df)
print(org_churn)
assert(org_churn == 2./4.)

# new customer
print("test get_new_customers")
org_new_customer_df = get_new_customers(test_df)
print(org_new_customer_df)
assert (1 == len(list([org_new_customer_df['net_sales'] != 0.])))

# count engaged or non engaged given a year
print("test get_count_engaged_customers")
cnt_engaged_18 = get_count_engaged_customers(test_df, 2018)
print(cnt_engaged_18)
assert(cnt_engaged_18 == 3)
cnt_engaged_19 = get_count_engaged_customers(test_df, 2019)
print(cnt_engaged_19)
assert(cnt_engaged_19 == 3)

# count non engaged given a year
print("test get_count_non_engaged_customers")
cnt_nonengaged_18 = get_count_non_engaged_customers(test_df, 2018)
print(cnt_nonengaged_18)
assert(cnt_nonengaged_18 == 1)
cnt_nonengaged_19 = get_count_non_engaged_customers(test_df, 2019)
print(cnt_nonengaged_19)
assert(cnt_nonengaged_19 == 0)

print("test get_avg_revenue_engaged")
avg_rev_eng = get_avg_revenue_engaged(test_df, 2018)
print(avg_rev_eng)
assert(avg_rev_eng == ((10+2+5+1)/3))

print("test get_avg_revenue_non_engaged")
avg_rev_noneng = get_avg_revenue_non_engaged(test_df, 2018)
print(avg_rev_noneng)
assert(avg_rev_noneng == ((25)/1))

# average rev for year will be diff than avg rev for combined years bec of retention

test org_retention
numerator 2
denominator 3
0.6666666666666666
numerator 0
denominator 1
0.0
test org_churn
0.5
test get_new_customers
  hh_id  hh_engaged  net_sales gl_shelf  fiscal_period  fiscal_year
9     F        True          5     APPL         201904         2019
test get_count_engaged_customers
3
3
test get_count_non_engaged_customers
1
0
test get_avg_revenue_engaged
6.0
test get_avg_revenue_non_engaged
25.0


In [12]:
### New Grower Revenue
# __Hypothesis__: Is there a positive difference between revenue from engaged new customers vs. non-engaged new customers
# - Sum(Revenue from New Customers who are Engaged)
#   - Revenue calculated YTD for only the current year
#
# New Customers: (2019 HH purchases == True) AND (2018 HH purchases == False)
# 
# Inputs: 
#  Dataset of sales totals for all engaged households and non-engaged households for 2018 and 2019
#    cust_ID	cn_engaged	net_sales	fiscal_period	fiscal_year
# 
# Outputs:
#  
# 
# Method:
#  Left Join sales by month by account on account number to get boolean indicator for HH digital engagement,
#    HH id and name

def new_grower_revenue(sales_df, engaged=True, for_year=2019):
    df = get_new_customers(sales_df, for_year)
    new_grower_sales = df.query('hh_engaged == @engaged')['net_sales'].sum()
    return new_grower_sales

In [13]:
print('testing new_grower_revenue with test_df')

# engaged 2019
print(new_grower_revenue(test_df))
assert(new_grower_revenue(test_df) == 5.)

# non-engaged 2019
print(new_grower_revenue(test_df, False))
assert(new_grower_revenue(test_df, False) == 0.)


testing new_grower_revenue with test_df
5
0


In [14]:
#### [Revenue] Benefit from Digital Customer Retention
### Benefit from Digital Customer Retention
# __Hypothesis__: Is there a positive trend to revenue benefit from digital engagement
# - Count(Engaged Customers) * Avg(Annual Sales Revenue for Engaged Customers) * %(Churn Non-Engaged - Churn Engaged)
#   - assume '%' means we want the percent benefit
#   - Sales revenue is calculated for the prior year
#
#### Churn
# org_churn_percent: Count(households with 2018 sales and no sales in 2019) / Count(households with 2018 sales) 
#
# Input: 
#  Dataset of sales totals for all engaged households and non-engaged households for 2018 and 2019
#    cust_ID, cn_engaged, net_sales, fiscal_period, fiscal_year
#
# Outputs:
#  1) Benefit: float, currency
# 
# Method:
#  count_engaged = count(engaged customers for ytd)
#  avg_engaged_rev = avg(engaged customer sales revenue for ytd)
#  churn_engaged = count(engaged households with sales != 0 in 2018 and sales == 0 in 2019) / 
#                                      count(engaged households with sales != 0 in 2018)
#  churn_non_engaged = count(non-engaged households with sales != 0 in 2018 and sales == 0 in 2019) / 
#                                      count(non-engaged households with sales != 0 in 2018)
#
#  Benefit = count_engaged * avg_engaged_rev * (churn_engaged - churn_non_engaged)
#
def benefit_digital_customer_retention(hh_sales_df, for_year=2019):
    # get_count_engaged_customers(df, for_year=2019)
    count_engaged_customers = get_count_engaged_customers(hh_sales_df, for_year)
    # get_avg_revenue_engaged for the prior year
    avg_ann_rev_eng = get_avg_revenue_engaged(hh_sales_df, for_year-1)
    # get_org_churn(df, curr_year=2019)
    org_churn_eng = get_org_churn(hh_sales_df.query('hh_engaged == True'), for_year)
    org_churn_not_eng = get_org_churn(hh_sales_df.query('hh_engaged == False'), for_year)
    
    return count_engaged_customers * avg_ann_rev_eng * (org_churn_not_eng - org_churn_eng)


In [15]:
print('${:,.2f}'.format(benefit_digital_customer_retention(test_df, 2019)))

# for the test data
# Count(Engaged Customers) for the current year
cnt_eng_19 = 3
# Avg(Annual Sales Revenue for Engaged Customers) for the prior year - 2018
avg_ann_rev_eng = (10+2+5+1)/3

# Churn Non-Engaged 
# Count(non-engaged Households with 2018 purchases but no 2019 purchases) / Count(non-engaged Households with 2018 purchases)
churn_non = 1/1
# Churn Engaged
churn_eng = 1/3

# Count(Engaged Customers) * Avg(Annual Sales Revenue for Engaged Customers) * %(Churn Non-Engaged - Churn Engaged)
res = cnt_eng_19 * avg_ann_rev_eng * (churn_non - churn_eng)
print(res)

assert(res == benefit_digital_customer_retention(test_df))

$12.00
12.000000000000002


In [16]:
# calc benefit iteratively 
count_engaged_customers = get_count_engaged_customers(test_df)
avg_ann_rev_eng = get_avg_revenue_engaged(test_df, 2018)
ann_rev_eng = get_revenue_engaged(test_df, 2018)
org_churn_eng = get_org_churn(test_df.query('hh_engaged == True'))
org_churn_not_eng = get_org_churn(test_df.query('hh_engaged == False'))

print('count_engaged_customers: {}'.format(count_engaged_customers))
print('avg_ann_rev_eng: ${:,.2f}'.format(avg_ann_rev_eng))
print('ann_rev_eng: ${:,.2f}'.format(ann_rev_eng))
print('org_churn_eng: {:.2f}'.format(org_churn_eng))
print('org_churn_not_eng: {:.2f}'.format(org_churn_not_eng))

benefit_from_digital_retention = count_engaged_customers * avg_ann_rev_eng *(org_churn_not_eng - org_churn_eng)
print('${:,.2f}'.format(benefit_from_digital_retention))
# same as ann_rev since count_engaged_customers cancel out:
print('${:,.2f}'.format(ann_rev_eng * (org_churn_not_eng - org_churn_eng)))

count_engaged_customers: 3
avg_ann_rev_eng: $6.00
ann_rev_eng: $18.00
org_churn_eng: 0.33
org_churn_not_eng: 1.00
$12.00
$12.00


In [17]:
#### Increased Customer Retention
# (Engaged Customer Retention - NON-Engaged Customer Retention)
#
# Input: 
#  hh_sales_df: Dataset of sales totals for all engaged households and non-engaged households for 2018 and 2019
#    columns: cust_ID, cn_engaged, net_sales, fiscal_period, fiscal_year
#  
# Outputs:
#  inc_customer_retention: float, proportion
# 
# Method:
#  retention_engaged = count(engaged distinct households with sales != 0 in 2018 and sales != 0 in 2019) / 
#                                      count(engaged distinct households with sales != 0 in 2018)
# 
#  retention_non_engaged = count(non-engaged distinct households with sales != 0 in 2018 and sales != 0 in 2019) / 
#                                      count(non-engaged disctinct households with sales != 0 in 2018)
# 
#  inc_customer_retention = retention_engaged - retention_non_engaged

def inc_customer_retention(hh_df, for_year=2019, verbose=False):
    retention_engaged = get_org_retention(hh_df, for_year, engaged=True, verbose=verbose)
    if verbose:
        print('retention_engaged {}'.format(retention_engaged))

    retention_non_engaged = get_org_retention(hh_df, for_year, engaged=False, verbose=verbose)
    if verbose:
        print('retention_non_engaged {}'.format(retention_non_engaged))

    if verbose:
        print(retention_engaged - retention_non_engaged)
        
    return (retention_engaged - retention_non_engaged)

In [18]:
print(inc_customer_retention(test_df))
print((2/3. - 0/1.))
assert(inc_customer_retention(test_df) == (2/3. - 0/1.))

0.6666666666666666
0.6666666666666666


In [19]:
#### Increased Share of Wallet
# Avg(Annual Revenue for Engaged Customers) - Avg(Annual Revenue for NON-Engaged Customers)
# 
# Input: 
#  hh_sales_df: Dataset of sales totals for all engaged households and non-engaged households for 2018 and 2019
#    columns: cust_ID, cn_engaged, net_sales, fiscal_period, fiscal_year
#
# Outputs:
#   1) inc_share_of_wallet: float (currency)
#
# Method:
#  avg_engaged_rev = avg(engaged customer sales revenue for selected year)
#  avg_non_engaged_rev = avg(non-engaged customer sales revenue for selected year)
#
def inc_share_of_wallet(hh_sales_df, for_year=2019, verbose=False):
    avg_revenue_engaged = get_avg_revenue_engaged(hh_sales_df, for_year)
    if verbose:
        print(avg_revenue_engaged)
        
    avg_revenue_non_engaged = get_avg_revenue_non_engaged(hh_sales_df, for_year)
    if verbose:
        print(avg_revenue_non_engaged)
        
    return (avg_revenue_engaged - avg_revenue_non_engaged)

In [20]:
print('inc_share_of_wallet from engagement 2018: ${:,.2f}'.format(inc_share_of_wallet(test_df, 2018, True)))
print('inc_share_of_wallet from engagement 2019: ${:,.2f}'.format(inc_share_of_wallet(test_df, 2019, True)))

6.0
25.0
inc_share_of_wallet from engagement 2018: $-19.00
4.0
0
inc_share_of_wallet from engagement 2019: $4.00


In [21]:
# Increased Agronomic Services Revenue, Engaged and Non-Engaged
# "/users/kmatson/documents/nutrien-digital/sales_acct_hh_appl_engaged_df.pickle"

# Hypothesis: Does the Nutrien "buy" for Agronomic Services increase as a result of engagement?

# YoY, YTD Comparison of Sum(Application Services Revenue for Engaged Customers)
# YoY, YTD Comparison of Sum(Application Services Revenue for NON-Engaged Customers)
# requires shelf-level sales data

def agronomic_services_revenue(hh_sales_df, for_year=2019, engaged=True):
    # strictly request gl_shelf for APPL
    result = hh_sales_df.query('fiscal_year == @for_year and hh_engaged == @engaged and gl_shelf == "APPL" and net_sales != 0')['net_sales'].sum()
    
    return result

In [22]:
engaged_2018_agronomic_services_revenue = (agronomic_services_revenue(test_df, 2018, True))
NON_engaged_2018_agronomic_services_revenue = (agronomic_services_revenue(test_df, 2018, False))
engaged_2019_agronomic_services_revenue = (agronomic_services_revenue(test_df, 2019, True))
NON_engaged_2019_agronomic_services_revenue = (agronomic_services_revenue(test_df, 2019, False))

print('2018 engaged agronomic_services_revenue:     ${:,.2f}'.format(engaged_2018_agronomic_services_revenue))
print('2018 NON-engaged agronomic_services_revenue: ${:,.2f}'.format(NON_engaged_2018_agronomic_services_revenue))
print('2019 engaged agronomic_services_revenue:     ${:,.2f}'.format(engaged_2019_agronomic_services_revenue))
print('2019 NON-engaged agronomic_services_revenue: ${:,.2f}'.format(NON_engaged_2019_agronomic_services_revenue))

assert(engaged_2018_agronomic_services_revenue == 2+5.)
assert(NON_engaged_2018_agronomic_services_revenue == 25.)
assert(engaged_2019_agronomic_services_revenue == 5+5.)
assert(NON_engaged_2019_agronomic_services_revenue == 0.)


2018 engaged agronomic_services_revenue:     $7.00
2018 NON-engaged agronomic_services_revenue: $25.00
2019 engaged agronomic_services_revenue:     $10.00
2019 NON-engaged agronomic_services_revenue: $0.00


In [23]:
# if output needed
# test_df.to_csv("/users/kmatson/documents/nutrien-digital/dvr_simple_test.csv")

## Results

The production data gives the following results:

In [24]:
# eng_sales_pkl = "/users/kmatson/documents/nutrien-digital/sales_hh_acct_fp_engaged_df.pickle"
eng_sales_pkl = "C:\\Users\\kmatson.AGRIUM\\iCloudDrive\\Documents\\nutrien-digital\\sales_hh_fp_engaged_df.pickle"

eng_sales_df = un_pickle(eng_sales_pkl)
eng_sales_df.head()

,fiscal_year,fiscal_period,hh_id,gl_shelf,net_sales,acct_id,person_return_visits,hh_engaged,is_digital,_merge
0,2018,201801,AH_1001007,APPL,0.00,1001007,-1.0,False,False,both
1,2018,201801,AH_1001007,FERT,8482.56,1001007,-1.0,False,False,both
2,2018,201801,AH_1001014,APPL,78.30,1001014,26.0,True,True,both
3,2018,201801,AH_1001014,CHEM,1241.17,1001014,26.0,True,True,both
4,2018,201801,AH_1001057,CHEM,658.10,1001057,13.0,True,True,both


### New Grower Revenue, Digitally Engaged

In [25]:
print('New Grower Revenue, Digitally Engaged: ${:,.2f}'.format(new_grower_revenue(eng_sales_df)))
print('New Grower Revenue, Non-Digitally Engaged: ${:,.2f}'.format(new_grower_revenue(eng_sales_df, False)))

New Grower Revenue, Digitally Engaged: $50,167,301.74
New Grower Revenue, Non-Digitally Engaged: $621,369,606.59


### Revenue Benefit from Digital Customer Retention

In [26]:
print('Revenue benefit of digital customer retention 2019: ${:,.2f}'\
      .format(benefit_digital_customer_retention(eng_sales_df, 2019)))

Revenue benefit of digital customer retention 2019: $677,788,873.83


### Increased Customer Retention from Digital

In [27]:
print('Increased Customer Retention from Digital: {:.2f}%'.format(100*inc_customer_retention(eng_sales_df, for_year=2019)))

Increased Customer Retention from Digital: 20.01%


### Increased Share of Wallet from Digital Engagement

In [28]:
print('Increased share of wallet from Digital Engagement: ${:,.2f}'.format(inc_share_of_wallet(eng_sales_df)))

Increased share of wallet from Digital Engagement: $143,793.61


### Agronomic Services Revenue

In [32]:
ag_svc_rev_eng_2019 = agronomic_services_revenue(eng_sales_df, engaged=True)
ag_svc_rev_noneng_2019 = agronomic_services_revenue(eng_sales_df, engaged=False)
print("ag services revenue 2019 \t\t\t${:,.2f}".format(ag_svc_rev_eng_2019))
print("ag services revenue 2019 Non-Engaged \t\t${:,.2f}".format(ag_svc_rev_noneng_2019))
print("ag services revenue 2019 Engaged v. Non-Engaged ${:,.2f}".format(ag_svc_rev_eng_2019 - ag_svc_rev_noneng_2019))

ag services revenue 2019 			$60,400,596.78
ag services revenue 2019 Non-Engaged 		$172,387,706.24
ag services revenue 2019 Engaged v. Non-Engaged $-111,987,109.46


In [33]:
ag_svc_rev_eng_2018 = agronomic_services_revenue(eng_sales_df, 2018, True)
ag_svc_rev_noneng_2018 = agronomic_services_revenue(eng_sales_df, 2018, False)
print("ag services revenue 2018 \t\t\t${:,.2f}".format(ag_svc_rev_eng_2018))
print("ag services revenue 2018 Non-Engaged \t\t${:,.2f}".format(ag_svc_rev_noneng_2018))
print("ag services revenue 2018 Engaged v. Non-Engaged ${:,.2f}".format(ag_svc_rev_eng_2018 - ag_svc_rev_noneng_2018))

ag services revenue 2018 			$92,185,316.24
ag services revenue 2018 Non-Engaged 		$232,732,890.96
ag services revenue 2018 Engaged v. Non-Engaged $-140,547,574.72


### Explore Retention

- Retention: Count(Households with 2018 purchases who ALSO have 2019 purchases) / Count(Households with 2018 purchases)
- ==> Count(Households with prior year purchases who ALSO have current year purchases) / Count(Households with prior year purchases)
  - Purchases for current year are YTD based on records to-date in the data set 
  - Retention is expected to __increase__ throughout the current year as prior year accounts for a household are activated (bought against) in the current year

In [34]:
# explore retention results
org_retention_engaged = get_org_retention(eng_sales_df, for_year=2019, engaged=True, verbose=False)
org_retention_non_engaged = get_org_retention(eng_sales_df, for_year=2019, engaged=False, verbose=False)
print('2018 to 2019 Nutrien Customer Retention for Digitally Engaged Customers {:.1f}%'.format(100*org_retention_engaged))
print('2018 to 2019 Nutrien Customer Retention for Non-Engaged Customers       {:.1f}%'.format(100*org_retention_non_engaged))

2018 to 2019 Nutrien Customer Retention for Digitally Engaged Customers 87.8%
2018 to 2019 Nutrien Customer Retention for Non-Engaged Customers       67.8%


### Churn

- Churn: Count(Households with prior yr purchases but no current yr purchases) / Count(Households with prior year purchases)
  - Purchases for current year are YTD based on records to-date in the data set 
  - Churn is expected to __decrease__ over the current year as prior year accounts get activated in the current year
  - Churn of engaged customers is expected to be __less than__ churn from non-engaged customers

In [35]:
org_churn = get_org_churn(eng_sales_df)
org_churn_engaged = get_org_churn(eng_sales_df.query('hh_engaged==True'))
org_churn_non_engaged = get_org_churn(eng_sales_df.query('hh_engaged==False'))
print('2018 to 2019 Nutrien Customer Churn                                 {:.1f}%'.format(100*org_churn))
print('2018 to 2019 Nutrien Customer Churn for Digitally Engaged Customers {:.1f}%'.format(100*org_churn_engaged))
print('2018 to 2019 Nutrien Customer Churn for Non-Engaged Customers       {:.1f}%'.format(100*org_churn_non_engaged))

2018 to 2019 Nutrien Customer Churn                                 30.5%
2018 to 2019 Nutrien Customer Churn for Digitally Engaged Customers 12.2%
2018 to 2019 Nutrien Customer Churn for Non-Engaged Customers       32.2%


### Churn by Revenue Category

In [ ]:
# churn calculations for revenue groupings
# create df for annual sum by account for 2018
# assign the category attribute to each account 
# extract the revenue category for each acct/hh_id and merge with the sales df 
# calculate churn for each category

In [50]:
#cat_sales_pkl = "C:\\Users\\kmatson.AGRIUM\\iCloudDrive\\Documents\\nutrien-digital\\sales_hh_fy_rev_summary_df.pickle"
# cat_sales_df = un_pickle(cat_sales_pkl)
# cat_sales_df.head()

In [65]:
# get the categories from the 2018 data
eng_sales18_df = pd.DataFrame(eng_sales_df.query('fiscal_year == 2018').\
                                        pivot_table(index=['fiscal_year','hh_id','is_digital','hh_engaged'], \
                                        values=['net_sales'], aggfunc='sum').to_records())

eng_sales18_df['rev_bins'] = pd.cut(eng_sales18_df['net_sales'], \
                                              bins=[eng_sales18_df['net_sales'].min(),\
                                                    0,1000.,5000.,30000.,100000.,eng_sales18_df['net_sales'].max()],\
                                              labels=['< $0','$0-1K','$1-5K','$5-30K','$30-100K','> $100K'])
pd.options.display.float_format = '${:,.0f}'.format
eng_sales18_df.pivot_table(index=['rev_bins'], \
                                        values=['net_sales'], aggfunc=['count','sum','mean','median'])

,count,sum,mean,median
,net_sales,net_sales,net_sales,net_sales
rev_bins,,,,
< $0,2754,"$-3,790,765","$-1,376",$0
$0-1K,51567,"$18,379,946",$356,$280
$1-5K,45049,"$114,825,936","$2,549","$2,325"
$5-30K,55177,"$765,666,529","$13,877","$12,197"
$30-100K,30806,"$1,730,882,356","$56,187","$52,106"
> $100K,21727,"$7,295,646,941","$335,787","$187,714"


In [66]:
eng_sales18_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207081 entries, 0 to 207080
Data columns (total 6 columns):
fiscal_year    207081 non-null int32
hh_id          207081 non-null object
is_digital     207081 non-null bool
hh_engaged     207081 non-null bool
net_sales      207081 non-null float64
rev_bins       207080 non-null category
dtypes: bool(2), category(1), float64(1), int32(1), object(1)
memory usage: 4.5+ MB


In [69]:
# create a category LUT for accounts
eng_sales18_cats_df = pd.DataFrame(eng_sales18_df.pivot_table(index=['hh_id','rev_bins']).to_records())
eng_sales18_cats_df.head()

,hh_id,rev_bins,fiscal_year,hh_engaged,is_digital,net_sales
0,AH_1001004,$5-30K,2018,False,True,"$27,948"
1,AH_1001007,> $100K,2018,False,False,"$130,419"
2,AH_1001008,$5-30K,2018,True,True,"$8,457"
3,AH_1001013,$1-5K,2018,False,False,"$2,462"
4,AH_1001014,> $100K,2018,True,True,"$127,972"


In [71]:
eng_sales18_cats_df = eng_sales18_cats_df.drop(['fiscal_year','hh_engaged','is_digital','net_sales'], axis=1).copy()
eng_sales18_cats_df.head()

,hh_id,rev_bins
0,AH_1001004,$5-30K
1,AH_1001007,> $100K
2,AH_1001008,$5-30K
3,AH_1001013,$1-5K
4,AH_1001014,> $100K


In [72]:
cat_sales_df = eng_sales_df.merge(eng_sales18_cats_df, how='left', on='hh_id')

In [82]:
# for each category get the churn
def get_churn_by_cat(df):
    org_churn = get_org_churn(df)
    org_churn_engaged = get_org_churn(df.query('hh_engaged==True'))
    org_churn_non_engaged = get_org_churn(df.query('hh_engaged==False'))
    print('{:.1f}% {:.1f}% {:.1f}%'.format(100*org_churn, 100*org_churn_engaged, 100*org_churn_non_engaged))

In [83]:
print('Overall Digitally-Engaged Non-Engaged')
for name, group in cat_sales_df.groupby('rev_bins'): 
    # print the name of the regiment
    print(name)
    # print the data of that regiment
    get_churn_by_cat(group)

Overall Digitally-Engaged Non-Engaged
$0-1K
53.5% 29.2% 53.8%
$1-5K
33.9% 21.3% 34.2%
$30-100K
14.9% 13.0% 15.2%
$5-30K
22.7% 16.4% 23.2%
< $0
72.9% 47.8% 74.5%
> $100K
7.7% 6.1% 8.5%


In [84]:
#overall
get_churn_by_cat(cat_sales_df)

30.5% 12.2% 32.2%


#### New Customer:
- New Customer: (Households with 2019 account purchases but no 2018 account purchases)
  - Purchases for current year are YTD based on records to-date in the data set 
  - The new cusotmer count is expected to __increase__ over the current year as new households are acquired in the current year


#### Test Data


In [36]:
# simple test data
# todo: add NaNs to see where fails
test_data2 = {'hh_id': ['A', 'B', 'B', 'C', 'D', 'E', 'B', 'C'], 
        'hh_engaged': [False, True, True, True, False, True, True, False], 
        'net_sales': [5, 10, 2, 5, 25, 1, 5, 2],
        'fiscal_year':[2018, 2018, 2018, 2018, 2018, 2019, 2019, 2019]}
test_df2 = pd.DataFrame(test_data2)
test_df2

,hh_id,hh_engaged,net_sales,fiscal_year
0,A,False,5,2018
1,B,True,10,2018
2,B,True,2,2018
3,C,True,5,2018
4,D,False,25,2018
5,E,True,1,2019
6,B,True,5,2019
7,C,False,2,2019


In [37]:
print(get_count_engaged_customers(test_df2))
print(get_count_non_engaged_customers(test_df2))

print(get_avg_revenue_non_engaged(test_df2, for_year=2018) == (5+25)/2)
print(get_avg_revenue_engaged(test_df2, for_year=2018) == (10+2+5)/2)

print(get_avg_revenue_non_engaged(test_df2, for_year=2019) == (2)/1)
print(get_avg_revenue_engaged(test_df2, for_year=2019) == (1+5)/2)

print((5+25)/2)
print((10+2+5)/2)

2
1
True
True
True
True
15.0
8.5
